In [1]:
import os
import io
import json
import argparse
import torch

from transformers import StoppingCriteria, StoppingCriteriaList

from PIL import Image
import numpy as np
import numpy as np

import torchvision.transforms as T

from torchvision.transforms.functional import InterpolationMode
from torch.utils.data import Dataset

from torchvision import transforms

import matplotlib.pyplot as plt

from tqdm import tqdm

from IPython.display import Video, HTML

from decord import VideoReader, cpu

/gs/home/lihongyi/anaconda3/envs/llama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def save_arguments(args, filepath):
    with open(filepath, 'w') as file:
        json.dump(vars(args), file)

def load_arguments(filepath):
    with open(filepath, 'r') as file:
        args_dict = json.load(file)
    return args_dict

# Optionally, repopulate argparse.ArgumentParser with these arguments
def repopulate_arguments(args_dict):
    parser = argparse.ArgumentParser(description="Example script")
    for key, value in args_dict.items():
        parser.add_argument(f'--{key}', type=type(value),default=value)
    return parser.parse_args([])

In [5]:
path = '../vqa_checkpoint/checkpoint_pretrain/llama2_13b_acc8_br8e3_bs4_vnips'

loaded_args = load_arguments(path+'/args.json')

args = repopulate_arguments(loaded_args)
args.llama_model_path = '.' +args.llama_model_path
args.resume='../vqa_checkpoint/checkpoint_pretrain/llama2_13b_acc8_br8e3_bs4_vnips/checkpoint_18.pth'


# path = '../vqa_checkpoint/checkpoint_pretrain/llama2_7b_acc4_br5e3_correct_vnips'

# loaded_args = load_arguments(path+'/args.json')

# args = repopulate_arguments(loaded_args)
# args.llama_model_path = '.' +args.llama_model_path
# args.resume='../../vqa_checkpoint/checkpoint_pretrain/llama2_7b_acc4_br5e3_correct_vnips/checkpoint_19.pth'

import sys
sys.path.append('../')
from llama import Tokenizer
from llama_vqa import LLaMA_VQA
from dataloader import load_data

with torch.no_grad():
    model = LLaMA_VQA(args)

checkpoint = torch.load(args.resume, map_location='cpu')
model.load_state_dict(checkpoint['model'], strict=False)
tokenizer = Tokenizer(model_path=f'{args.llama_model_path}./tokenizer.model')

Using model: 13B
loading from ../pretrained/llama2/13B/consolidated.00.pth
loading from ../pretrained/llama2/13B/consolidated.01.pth
gathering layer 0 of 40
gathering layer 1 of 40
gathering layer 2 of 40
gathering layer 3 of 40
gathering layer 4 of 40
gathering layer 5 of 40
gathering layer 6 of 40
gathering layer 7 of 40
gathering layer 8 of 40
gathering layer 9 of 40
gathering layer 10 of 40
gathering layer 11 of 40
gathering layer 12 of 40
gathering layer 13 of 40
gathering layer 14 of 40
gathering layer 15 of 40
gathering layer 16 of 40
gathering layer 17 of 40
gathering layer 18 of 40
gathering layer 19 of 40
gathering layer 20 of 40
gathering layer 21 of 40
gathering layer 22 of 40
gathering layer 23 of 40
gathering layer 24 of 40
gathering layer 25 of 40
gathering layer 26 of 40
gathering layer 27 of 40
gathering layer 28 of 40
gathering layer 29 of 40
gathering layer 30 of 40
gathering layer 31 of 40
gathering layer 32 of 40
gathering layer 33 of 40
gathering layer 34 of 40
ga

OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB. GPU 0 has a total capacty of 39.59 GiB of which 2.19 MiB is free. Including non-PyTorch memory, this process has 39.58 GiB memory in use. Of the allocated memory 38.47 GiB is allocated by PyTorch, and 133.57 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [6]:
# args.resume='../../vqa_checkpoint/checkpoint_finetune/tag_vnip_llama2_13b_finetune_nextqa_2e3_acc16/checkpoint_4.pth'
# args.resume='../../vqa_checkpoint/checkpoint_finetune/tag_vnip_llama2_13b_finetune_star_2e3_acc16/checkpoint_1.pth'
# args.resume='../../vqa_checkpoint/checkpoint_pretrain/llama2_13b_acc8_br8e3_bs4_vnips/checkpoint_18.pth'
# args.llama_model_path = '.' +args.llama_model_path
# checkpoint = torch.load(args.resume, map_location='cpu')
# model.load_state_dict(checkpoint['model'], strict=False)
# tokenizer = Tokenizer(model_path=f'{args.llama_model_path}./tokenizer.model')

In [7]:
model = model.eval()

In [11]:
def mutli_choice_decoding(model, tokenizer, prompt1,prompt2,video=None):
    adapter = model.adapter_query.weight.reshape(-1, model.adapter_len, model.params.dim).unsqueeze(1)
    freqs= model.freqs_cis.cuda()
    
    tokens = [tokenizer.bos_id] + tokenizer.sp_model.encode(prompt1)
    query = torch.tensor(tokens, dtype=torch.int64).cuda()
    input_embedding = model.tok_embeddings(query)

    tokens_2 = tokenizer.sp_model.encode(prompt2)
    query_2 = torch.tensor(tokens_2, dtype=torch.int64).cuda()
    input_embedding_2 = model.tok_embeddings(query_2)
    tokens.extend(tokens_2)
    video = video.cuda().float()
    video/=video.norm(dim=-1,keepdim=True)
    if True:
        sim = video@model.memory.T

        sim = (sim*100).softmax(dim=-1)

        video = sim@model.memory
        video = video/video.norm(dim=-1,keepdim=True)
        
    video_feature = model.visual_proj(video)
    video_feature = (video_feature + model.temporal_emb.weight[:, :]).type(model.llamatype)
    vqa_video_start=input_embedding.shape[0]
    # print(video_feature.shape)
    input_embedding = torch.cat([input_embedding,video_feature,input_embedding_2])
    start_pos=0
    for j in range(2):
        vqa_h = input_embedding.unsqueeze(0)
        seqlen = vqa_h.shape[-2]
        freqs_cis = freqs[:seqlen]
        mask = None
        mask = torch.full((1, 1, seqlen, seqlen), float("-inf"), device=vqa_h.device)
        mask = torch.triu(mask, diagonal=0 + 1).type_as(vqa_h)

        for i, layer in enumerate(model.layers[-1 * model.adapter_layer:]):
            vqa_h = layer(vqa_h, start_pos, freqs_cis, mask, adapter[i].type(model.llamatype), vqa_video_start)
        vqa_h = model.norm(vqa_h)
        vqa_output = model.output(vqa_h)
        vqa_output = vqa_output.reshape(-1, model.vocab_size)
        vqa_output[-1,920]=-100
        vqa_output[-1,1128]=-100
        next_token = vqa_output[-1,:].argmax()
        tokens.append(next_token.item())
        token_emb = model.tok_embeddings(next_token.unsqueeze(0))
        input_embedding = torch.cat([input_embedding,token_emb],dim=0)
    return tokens

In [12]:
your_data_path = "../../mvbench/MVBench/video"
data_list = {
    "Action Sequence": ("action_sequence.json", f"{your_data_path}/star/Charades_v1_480/", "video", True), # has start & end
    "Action Prediction": ("action_prediction.json", f"{your_data_path}/star/Charades_v1_480/", "video", True), # has start & end
    "Action Antonym": ("action_antonym.json", f"{your_data_path}/ssv2_video/", "video", False),
    "Fine-grained Action": ("fine_grained_action.json", f"{your_data_path}/Moments_in_Time_Raw/videos/", "video", False),
    "Unexpected Action": ("unexpected_action.json", f"{your_data_path}/FunQA_test/test/", "video", False),
    "Object Existence": ("object_existence.json", f"{your_data_path}/clevrer/video_validation/", "video", False),
    "Object Interaction": ("object_interaction.json", f"{your_data_path}/star/Charades_v1_480/", "video", True), # has start & end
    "Object Shuffle": ("object_shuffle.json", f"{your_data_path}/perception/videos/", "video", False),
    "Moving Direction": ("moving_direction.json", f"{your_data_path}/clevrer/video_validation/", "video", False),
    "Action Localization": ("action_localization.json", f"{your_data_path}/sta/sta_video/", "video", True),  # has start & end
    "Scene Transition": ("scene_transition.json", f"{your_data_path}/scene_qa/video/", "video", False),
    "Action Count": ("action_count.json", f"{your_data_path}/perception/videos/", "video", False),
    "Moving Count": ("moving_count.json", f"{your_data_path}/clevrer/video_validation/", "video", False),
    "Moving Attribute": ("moving_attribute.json", f"{your_data_path}/clevrer/video_validation/", "video", False),
    "State Change": ("state_change.json", f"{your_data_path}/perception/videos/", "video", False),
    "Fine-grained Pose": ("fine_grained_pose.json", f"{your_data_path}/nturgbd/", "video", False),
    "Character Order": ("character_order.json", f"{your_data_path}/perception/videos/", "video", False),
    "Egocentric Navigation": ("egocentric_navigation.json", f"{your_data_path}/vlnqa/", "video", False),
    "Episodic Reasoning": ("episodic_reasoning.json", f"{your_data_path}/tvqa/frames_fps3_hq/", "frame", True),  # has start & end, read frame
    "Counterfactual Inference": ("counterfactual_inference.json", f"{your_data_path}/clevrer/video_validation/", "video", False),
}

data_dir = "../../mvbench/MVBench/json"

In [13]:
import clip
clip_model, preprocess = clip.load("ViT-L/14")

In [14]:
clip_model=clip_model.eval()

In [15]:
import cv2
from video_transforms import (
    GroupNormalize, GroupScale, GroupCenterCrop, 
    Stack, ToTorchFormatTensor
)

class MVBench_dataset(Dataset):
    def __init__(self, data_dir, data_list, num_segments=8, resolution=224):
        self.data_list = []
        for k, v in data_list.items():
            with open(os.path.join(data_dir, v[0]), 'r') as f:
                json_data = json.load(f)
            for data in json_data:
                self.data_list.append({
                    'task_type': k,
                    'prefix': v[1],
                    'data_type': v[2],
                    'bound': v[3],
                    'data': data
                })
        
        self.decord_method = {
            'video': self.read_video,
            'gif': self.read_gif,
            'frame': self.read_frame,
        }
        
        self.num_segments = num_segments
        
        # transform
        crop_size = resolution
        scale_size = resolution
        input_mean = [0.48145466, 0.4578275, 0.40821073]
        input_std = [0.26862954, 0.26130258, 0.27577711]
        self.transform = preprocess
    
    def __str__(self):
        len_list = {}
        option_list = {}
        for data in self.data_list:
            if data['task_type'] not in len_list:
                len_list[data['task_type']] = 0
            len_list[data['task_type']] += 1
            if data['task_type'] not in option_list:
                option_list[data['task_type']] = 0
            option_list[data['task_type']] += len(data['data']['candidates'])
        
        correct = 0
        total = 0
        res = f"There are {len(self.data_list)} videos as follow:\n"
        for k, v in len_list.items():
            correct += len_list[k]
            total += option_list[k]
            res += f"{v} for {k} ({option_list[k]} options => {len_list[k]/option_list[k]*100:.2f}%)\n"
            correct = correct + 1 / option_list[k]
        res += f"Total random accuracy: {correct/total*100:.2f}%"
        return res.rstrip()
        
    def __len__(self):
        return len(self.data_list)
    
    def get_index(self, bound, fps, max_frame, first_idx=0):
        if bound:
            start, end = bound[0], bound[1]
        else:
            start, end = -100000, 100000
        start_idx = max(first_idx, round(start * fps))
        end_idx = min(round(end * fps), max_frame)
        seg_size = float(end_idx - start_idx) / self.num_segments
        frame_indices = np.array([
            int(start_idx + (seg_size / 2) + np.round(seg_size * idx))
            for idx in range(self.num_segments)
        ])
        return frame_indices
    
    def read_video(self, video_path, bound=None):
        # cap = VideoReader(video_path, ctx=cpu(0), num_threads=1)
        cap = cv2.VideoCapture(video_path)
        max_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        # vr = cv2.VideoCapture(video_path)
        # max_frame = len(vr) - 1
        # fps = float(vr.get_avg_fps())
    
        images_group = list()
        frame_indices = self.get_index(bound, fps, max_frame, first_idx=0) 

        for frame_index in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
            ret, frame = cap.read()

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_image = Image.fromarray(frame)

            images_group.append(pil_image)


        # print(torch_imgs.shape)
        return images_group

    def read_gif(self, video_path, bound=None, fps=25):
        gif = imageio.get_reader(video_path)
        max_frame = len(gif) - 1
        
        images_group = list()
        frame_indices = self.get_index(bound, fps, max_frame, first_idx=0) 
        for index, frame in enumerate(gif):
            if index in frame_indices:
                img = cv2.cvtColor(frame, cv2.COLOR_RGBA2RGB)
                img = Image.fromarray(img)
                images_group.append(img)

        # print(torch_imgs.shape)
        return images_group
    
    def read_frame(self, video_path, bound=None, fps=3):
        max_frame = len(os.listdir(video_path))
        images_group = list()
        frame_indices = self.get_index(bound, fps, max_frame, first_idx=1) # frame_idx starts from 1
        for frame_index in frame_indices:
            img = Image.open(os.path.join(video_path, f"{frame_index:05d}.jpg"))
            images_group.append(img)
        # print(torch_imgs.shape)
        return images_group

    def qa_template(self, data):
        question = f"Question: {data['question']}\n"
        question += "Choices: \n"
        answer = data['answer']
        answer_idx = -1
        for idx, c in enumerate(data['candidates']):
            question += f"({chr(ord('A') + idx)}) {c}\n"
            if c == answer:
                answer_idx = idx
        question = question
        answer = f"({chr(ord('A') + answer_idx)}) {answer}"
        return question, answer

    def __getitem__(self, idx):

        decord_method = self.decord_method[self.data_list[idx]['data_type']]
        bound = None
        if self.data_list[idx]['bound']:
            bound = (
                self.data_list[idx]['data']['start'],
                self.data_list[idx]['data']['end'],
            )
        video_path = os.path.join(self.data_list[idx]['prefix'], self.data_list[idx]['data']['video'])
        question, answer = self.qa_template(self.data_list[idx]['data'])
        try:
            torch_imgs = decord_method(video_path, bound)
        except:
            return {
                'video': None, 
                'question': question, 
                'answer': answer,
                'task_type': self.data_list[idx]['task_type']
            }  
        # print(torch_imgs.shape)

            
        return {
            'video': torch_imgs, 
            'question': question, 
            'answer': answer,
            'task_type': self.data_list[idx]['task_type']
        }

In [16]:
#  position embedding
num_frame = 10
resolution = 224

dataset = MVBench_dataset(data_dir, data_list, num_segments=num_frame, resolution=resolution)

In [17]:
def check_ans(pred, gt):
    flag = False
    
    pred_list = pred.lower().split(' ')
    pred_option, pred_content = pred_list[0], ' '.join(pred_list[1:])
    gt_list = gt.lower().split(' ')
    gt_option, gt_content = gt_list[0], ' '.join(gt_list[1:])
    if gt_content[-1] == '.':
        gt_content = gt_content[:-1]
    
    if pred_option.replace('.', '') in gt_option:
        flag = True
    elif gt_option in pred_option:
        flag = True
        
    return flag

In [17]:
save_path = "./test"

correct = 0
total = 0
res_list = []
acc_dict = {}

for example in tqdm(dataset):
    task_type = example['task_type']
    if task_type not in acc_dict:
        acc_dict[task_type] = [0, 0] # correct, total
    acc_dict[task_type][1] += 1
    total += 1

    if example['video']==None:
        pred='(A)'
        print(1)
    else:
        prompt = "Instruction: Choose the correct answer based on the video and question.\nVideo:"
        prompt2 = f"\n{example['question']}Answer: The correct choice is ("
    
        tokens = mutli_choice_decoding(model,tokenizer,prompt,prompt2,example['video'])
        generate_text = tokenizer.decode(tokens[:])
        pred = generate_text.split('The correct choice is ')[1]


    gt = example['answer']
    res_list.append({
        'pred': pred,
        'gt': gt
    })
    if check_ans(pred=pred, gt=gt):
        acc_dict[task_type][0] += 1
        correct += 1
    if total%100==0:
        print(f"Part  Acc: {acc_dict[task_type][0] / acc_dict[task_type][1] * 100 :.2f}%")
        print(f"Total Acc: {correct / total * 100 :.2f}%")
        print('-' * 30, task_type, '-' * 30)
    # break

with open(f"{save_path}.json", "w") as f:
    json.dump({
        "acc_dict": acc_dict,
        "res_list": res_list
    }, f)

  2%|▎         | 100/4000 [00:58<29:05,  2.23it/s] 

Part  Acc: 43.00%
Total Acc: 43.00%
------------------------------ Action Sequence ------------------------------


  5%|▌         | 200/4000 [01:42<26:35,  2.38it/s]

Part  Acc: 42.00%
Total Acc: 42.00%
------------------------------ Action Sequence ------------------------------


  8%|▊         | 300/4000 [02:26<24:35,  2.51it/s]

...

Part  Acc: 49.00%
Total Acc: 40.29%
------------------------------ Episodic Reasoning ------------------------------


 98%|█████████▊| 3900/4000 [47:32<00:55,  1.81it/s]

Part  Acc: 32.00%
Total Acc: 40.08%
------------------------------ Counterfactual Inference ------------------------------


100%|██████████| 4000/4000 [48:30<00:00,  1.37it/s]

Part  Acc: 30.50%
Total Acc: 39.80%
------------------------------ Counterfactual Inference ------------------------------


In [18]:
def mutli_choice_decoding(model, tokenizer, prompt1,prompt2,video=None):
    with torch.no_grad():
        torch_imgs = [preprocess(image) for image in video]
        video = torch.stack(torch_imgs,dim=0)
        
        video = video.cuda()
        video = clip_model.encode_image(video)
        video/=video.norm(dim=-1,keepdim=True)
        adapter = model.adapter_query.weight.reshape(-1, model.adapter_len, model.params.dim).unsqueeze(1)
        freqs= model.freqs_cis.cuda()
        
        tokens = [tokenizer.bos_id] + tokenizer.sp_model.encode(prompt1)
        query = torch.tensor(tokens, dtype=torch.int64).cuda()
        input_embedding = model.tok_embeddings(query)
    
        tokens_2 = tokenizer.sp_model.encode(prompt2)
        query_2 = torch.tensor(tokens_2, dtype=torch.int64).cuda()
        input_embedding_2 = model.tok_embeddings(query_2)
        tokens.extend(tokens_2)
        video = video.cuda().float()
        video/=video.norm(dim=-1,keepdim=True)
        if True:
            sim = video@model.memory.T
    
            sim = (sim*100).softmax(dim=-1)
    
            video = sim@model.memory
            video = video/video.norm(dim=-1,keepdim=True)
            
        video_feature = model.visual_proj(video)
        video_feature = (video_feature + model.temporal_emb.weight[:, :]).type(model.llamatype)
        vqa_video_start=input_embedding.shape[0]
        # print(video_feature.shape)
        input_embedding = torch.cat([input_embedding,video_feature,input_embedding_2])
        start_pos=0
        for j in range(2):
            vqa_h = input_embedding.unsqueeze(0)
            seqlen = vqa_h.shape[-2]
            freqs_cis = freqs[:seqlen]
            mask = None
            mask = torch.full((1, 1, seqlen, seqlen), float("-inf"), device=vqa_h.device)
            mask = torch.triu(mask, diagonal=0 + 1).type_as(vqa_h)
    
            for i, layer in enumerate(model.layers[-1 * model.adapter_layer:]):
                vqa_h = layer(vqa_h, start_pos, freqs_cis, mask, adapter[i].type(model.llamatype), vqa_video_start)
            vqa_h = model.norm(vqa_h)
            vqa_output = model.output(vqa_h)
            vqa_output = vqa_output.reshape(-1, model.vocab_size)

            next_token = vqa_output[-1,:].argmax()
            tokens.append(next_token.item())
            token_emb = model.tok_embeddings(next_token.unsqueeze(0))
            input_embedding = torch.cat([input_embedding,token_emb],dim=0)
    return tokens

In [18]:
final_res = dict()
correct = 0
total = 0
for k, v in acc_dict.items():
    final_res[k] = v[0] / v[1] * 100
    correct += v[0]
    total += v[1]    
final_res['Avg'] = correct / total * 100

print(final_res)

with open("upload_leaderboard_7B_111.json", "w") as f:
    json.dump(final_res, f)

{'Action Sequence': 42.0, 'Action Prediction': 38.5, 'Action Antonym': 35.0, 'Fine-grained Action': 34.5, 'Unexpected Action': 66.0, 'Object Existence': 52.5, 'Object Interaction': 47.5, 'Object Shuffle': 28.000000000000004, 'Moving Direction': 22.0, 'Action Localization': 37.5, 'Scene Transition': 81.0, 'Action Count': 38.0, 'Moving Count': 24.0, 'Moving Attribute': 42.5, 'State Change': 41.5, 'Fine-grained Pose': 28.499999999999996, 'Character Order': 34.0, 'Egocentric Navigation': 23.5, 'Episodic Reasoning': 49.0, 'Counterfactual Inference': 30.5, 'Avg': 39.800000000000004}
